In [30]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._


In [31]:
!rm -r checkpoint


In [32]:
!mkdir checkpoint

In [33]:
!rm -r spark-warehouse

rm: cannot remove 'spark-warehouse': No such file or directory



In [34]:
spark.sql("set spark.sql.catalogImplementation").show()

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.catalog...| hive|
+--------------------+-----+



In [35]:
val df = spark.read
    .json("input/*.json")

df: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 17 more fields]


In [36]:
df.count()

res6: Long = 68


In [37]:
val schema = df.schema

schema: org.apache.spark.sql.types.StructType = StructType(StructField(agent-phone,StringType,true),StructField(bath,StringType,true),StructField(bedrooms,StringType,true),StructField(beds,StringType,true),StructField(data-est,StringType,true),StructField(data-lk,StringType,true),StructField(data-pk,StringType,true),StructField(data-pos,StringType,true),StructField(full_bathrooms,StringType,true),StructField(home_type,StringType,true),StructField(ldp-description-text,StringType,true),StructField(ldp-phone-link,StringType,true),StructField(price,StringType,true),StructField(property-info-address-citystatezip,StringType,true),StructField(property-info-address-main,StringType,true),StructField(sold date,StringType,true),StructField(sqft,StringType,true),StructField(status,StringType,true),...


In [38]:
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .option("cleanSource", "archive")
    .option("sourceArchiveDir", f"input_archive")
    .load("input/*.json")

df: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 17 more fields]


In [39]:
val df2 = df.writeStream
    .option("path", "output")
    .option("checkpointLocation", "checkpoint")
    .trigger(org.apache.spark.sql.streaming.Trigger.Once)
    .toTable("houses")

org.apache.spark.sql.AnalysisException:  org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [ ]:
df2.processAllAvailable
//df2.stop
df2.awaitTermination

In [18]:
val df3 = spark.read.format("jdbc").table("houses")

org.apache.spark.sql.AnalysisException:  org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [ ]:
df3.count

In [ ]:
df2.stop

In [13]:
val url = "jdbc:postgresql://localhost:5432/testing"


url: String = jdbc:postgresql://localhost:5432/testing


In [16]:
val connectionProperties = new Properties()

<console>: 35: error: not found: type Properties

In [15]:
connectionProperties.setProperty("Driver", "org.postgresql.Driver")

<console>: 37: error: not found: value connectionProperties

In [41]:
import java.sql.{Connection, DriverManager, ResultSet};

// Change to Your Database Config
val conn_str = "jdbc:mysql://localhost:5432/hive_metastore?user=postgres&password=xxx"

// Load the driver
//classOf[com.mysql.jdbc.Driver]

// Setup the connection
val conn = DriverManager.getConnection(conn_str)
try {
    // Configure to be Read Only
    val statement = conn.createStatement(ResultSet.TYPE_FORWARD_ONLY, ResultSet.CONCUR_READ_ONLY)

    // Execute Query
    val rs = statement.executeQuery("SELECT quote FROM quotes LIMIT 5")

    // Iterate Over ResultSet
    while (rs.next) {
        println(rs.getString("quote"))
    }
}
finally {
    conn.close
}

java.sql.SQLException:  No suitable driver found for jdbc:mysql://localhost:5432/hive_metastore?user=postgres&password=xxx

In [42]:
// Loading data from a JDBC source
val jdbcDF = spark.read
  .format("jdbc")
  .option("url", "jdbc:postgresql://localhost:5432/hive_metastore")
  .option("dbtable", "schema.test")
  .option("user", "postgres")
  .option("password", "xxx")
  .load()

org.postgresql.util.PSQLException:  Connection to localhost:5432 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections.